In [ ]:
import pyspark 
sc = pyspark.SparkContext('local[*]')

In [ ]:
import os
os.environ["HADOOP_USER_NAME"]="hdfs"

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
telemetry=sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('hdfs://172.17.0.3:9000/user/root/data/PdM_telemetry.csv')
errors=sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('hdfs://172.17.0.3:9000/user/root/data/PdM_errors.csv')
maint=sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('hdfs://172.17.0.3:9000/user/root/data/PdM_maint.csv')
failures=sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('hdfs://172.17.0.3:9000/user/root/data/PdM_failures.csv')
machines=sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load('hdfs://172.17.0.3:9000/user/root/data/PdM_machines.csv')

In [ ]:
import sys 

import lstmOps
import utilities_classifiersOps as classifiersOps

In [ ]:
import pandas as pd
df = pd.read_csv('dataset_24h.csv')

In [ ]:
features, labels, labelcolumns, trainScaler = classifiersOps.preprocessing_Multiclass(
    df, labelName=['failure'], dismissCols=['datetime', 'machineID'], dummies=True, scale=True)

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

spliter=StratifiedShuffleSplit(test_size=0.33)
train_indexes = []
test_indexes= []

for train_index, test_index in spliter.split(features, np.argmax(labels, axis=1)):
    train_indexes.append(train_index)
    test_indexes.append(test_index)

In [ ]:
x_train, y_train = features[train_indexes[-1]], labels[train_indexes[-1]]
x_test, y_test = features[test_indexes[-1]], labels[test_indexes[-1]]

In [ ]:
batched_feat, batched_label = lstmOps.split_sequences(x_train, y_train, 24)
batchedX_test, batchedY_test = lstmOps.split_sequences(x_test, y_test, 24)

In [ ]:
n_neurons=50
n_epoch=5

In [ ]:
features=batchedX_test

In [ ]:
scaler=1

In [ ]:
from copy import deepcopy
from sklearn.model_selection import train_test_split
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import datetime


In [ ]:
def create_model(stateful):
    model=Sequential()
    model.add(LSTM(50,
                   activation='relu',
                  input_shape=(24,31)))
    model.add(Dense(5))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [ ]:
def create_model(stateful):
    model=Sequential()
    model.add(LSTM(50,
                   activation='relu',
                  input_shape=(24,31)))
    model.add(Dense(5))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [ ]:
model_stateful = create_model(stateful=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
history=model_stateful.fit(batched_feat,batched_label,
                                          epochs=5,
                     batch_size=24)


In [ ]:
preds=model_stateful.predict(batchedX_test, batch_size=24)

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def invertScale(batchs_array, scaler):
    if len(batchs_array.shape) == 3:
        invert = []
        for i in range(batchs_array.shape[1]):
            inverted_batch = scaler.inverse_transform(batchs_array[:,i,:])
            inverted.appends(inverted_batch)
        return np.array(inverted).reshape(batchs_array.shape[0], batchs_array.shape[1], batchs_array.shape[2])
    else: 
        return scaler.inverse_transform(batchs_array)

In [ ]:
def make_predictions(model, features, scaler):
    preds=[]
    for t in range(features.shape[0]):
        pred = invertScale(model.predict(features[t].reshape(1,features[-1].shape[0],features[-1].shape[1])), scaler)
        preds.append(pred)
    preds = np.array(preds)
    preds = preds.reshape(preds.shape[0],preds.shape[2])
    return preds